In [97]:
import com.toldoven.aoc.notebook.AocClient
%use kandy
%use dataframe

val aoc = AocClient.fromEnv().interactiveDay(2025, 4)
aoc.viewPartOne()

val exampleInput = """
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.""".trim()
exampleInput.lines()

[..@@.@@@@., @@@.@.@.@@, @@@@@.@.@@, @.@@@@..@., @@.@@@@.@@, .@@@@@@@.@, .@.@.@.@@@, @.@@@.@@@@, .@@@@@@@@., @.@.@@@.@.]

In [98]:
import org.jetbrains.kotlinx.multik.api.mk
import org.jetbrains.kotlinx.multik.api.ndarray
import org.jetbrains.kotlinx.multik.ndarray.data.D2Array
import kotlin.streams.toList

val map: D2Array<Int> = mk.ndarray<Int>(exampleInput.lines().map { it.chars().toList() })
map

[[46, 46, 64, 64, 46, 64, 64, 64, 64, 46],
[64, 64, 64, 46, 64, 46, 64, 46, 64, 64],
[64, 64, 64, 64, 64, 46, 64, 46, 64, 64],
[64, 46, 64, 64, 64, 64, 46, 46, 64, 46],
[64, 64, 46, 64, 64, 64, 64, 46, 64, 64],
[46, 64, 64, 64, 64, 64, 64, 64, 46, 64],
[46, 64, 46, 64, 46, 64, 46, 64, 64, 64],
[64, 46, 64, 64, 64, 46, 64, 64, 64, 64],
[46, 64, 64, 64, 64, 64, 64, 64, 64, 46],
[64, 46, 64, 46, 64, 64, 64, 46, 64, 46]]

In [99]:
import org.jetbrains.kotlinx.multik.ndarray.data.D1
import org.jetbrains.kotlinx.multik.ndarray.data.D2Array
import org.jetbrains.kotlinx.multik.ndarray.data.MultiArray

fun MultiArray<Int, D1>.toLine() = toList().joinToString("") { it.toChar().toString() }

In [100]:
import org.jetbrains.kotlinx.multik.ndarray.data.get
import org.jetbrains.kotlinx.multik.ndarray.operations.forEachMultiIndexed
import org.jetbrains.kotlinx.multik.ndarray.operations.map
import org.jetbrains.kotlinx.multik.ndarray.operations.toList

val x = 2
val y = 1
val previousLine = if (y != 0) map[y - 1].toLine() else null
val nextLine = map[y + 1].toLine()
println("$previousLine\n$nextLine")



..@@.@@@@.
@@@@@.@.@@


In [101]:
import io.kotest.inspectors.shouldForAll
import io.kotest.matchers.shouldBe
import org.jetbrains.kotlinx.multik.ndarray.data.D2
import org.jetbrains.kotlinx.multik.ndarray.data.forEach
import org.jetbrains.kotlinx.multik.ndarray.operations.foldMultiIndexed


fun MultiArray<Int, D2>.getAdjacentIndices(row: Int, column: Int): List<Pair<Int, Int>> {
    return (row - 1..row + 1)
        .flatMap { r ->
            (column - 1..column + 1).map { c -> r to c }
        }
        .minus(row to column) // Remove item at row,column
        .filter { (r, c) -> r in 0 until shape[0] && c in 0 until shape[1] }
}

fun MultiArray<Int, D2>.getAdjacent(row: Int, column: Int): List<Char> {
//    val above = runCatching { get(row - 1, column - 1..column + 1) }.getOrNull()?.toList().orEmpty()
//    val below = runCatching { get(row - 2, column - 1..column + 1) }.getOrNull()?.toList().orEmpty()
//    val thisRow = runCatching { this[row, column - 1..column + 1] }.getOrNull()?.toList().orEmpty()
//    return (above + below + thisRow).map { it.toChar() }
    return getAdjacentIndices(row, column).mapNotNull { (r, c) -> this.get(r, c).toChar() }
}

val answer = """
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.""".trim().toMdArray()
val founds = mutableListOf<Pair<Int, Int>>()

// map[row,column]
fun String.toMdArray() = mk.ndarray<Int>(this.lines().map { it.chars().toList() })


fun solvePartOne(input: String): Int {
    val arr = input.toMdArray()
    return arr.foldMultiIndexed(0) { (row, column), count, value ->
        val adjacent = arr.getAdjacent(row, column)
        if (value.toChar() == '@' && adjacent.count { it == '@' } < 4) {
            founds.add(row to column)
            count + 1
        } else {
            count
        }
    }
}
founds.shouldForAll { (r, c) -> answer[r, c].toChar() shouldBe 'x' }
solvePartOne(exampleInput) shouldBe 13

13

In [103]:
aoc.submitPartOne(solvePartOne(aoc.input()))

Your answer: 1411. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [106]:
import org.jetbrains.kotlinx.multik.ndarray.data.set
import org.jetbrains.kotlinx.multik.ndarray.operations.filterMultiIndexed

fun getRemovable(arr: D2Array<Int>): List<Pair<Int, Int>> {
    val toRemove = mutableListOf<Pair<Int, Int>>()
    arr.forEachMultiIndexed { (row, column), value ->
        val adjacent = arr.getAdjacent(row, column)
        if (value.toChar() == '@' && adjacent.count { it == '@' } < 4) {
            toRemove.add(row to column)
        }
    }
    return toRemove
}

fun removePapers(arr: D2Array<Int>): Int {
    val toRemove = getRemovable(arr)
    return if (toRemove.isEmpty()) {
        0
    } else {
        val new = arr.deepCopy().also {
            toRemove.forEach { (r, c) -> it[r, c] = '.'.code }
        }
        toRemove.size + removePapers(new)
    }
}

fun solvePartTwo(input: String): Int {
    val arr = input.toMdArray()
    return removePapers(arr)
}

In [108]:
solvePartTwo(exampleInput) shouldBe 43
aoc.submitPartTwo(solvePartTwo(aoc.input()))

Your answer: 8557. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 4! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .